## Step 1: Tokenization

In [1]:
# Use the tiktokenizer to create tokens
with open("the-verdict.txt") as f:
    raw_text = f.read()

print(f"Number of characters in the file: {len(raw_text)}")

Number of characters in the file: 20479


In [2]:
# We will use tiktoken, which was used by GPT2
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [3]:
# Initialzie tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
# Encode the text from "the-verdict.txt" using tokenizer
integers = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
print(len(integers))

5145


In [5]:
# Generate back the string and compare with original raw text
strings = tokenizer.decode(integers)
print(len(strings))
print(len(strings) == len(raw_text))

20479
True


In [6]:
# Let's see how it decodes the unknown words
integers = tokenizer.encode("Akwirw ier")
print(integers)
print([tokenizer.decode([i]) for i in integers])
print(tokenizer.decode(integers))
    

[33901, 86, 343, 86, 220, 959]
['Ak', 'w', 'ir', 'w', ' ', 'ier']
Akwirw ier


### Step 2: CREATE INPUT-TARGET PAIRS USING DATALOADERS

##### STEPS:
<li>Tokenize entire text</li>
<li>Use a sliding window to chunk the book into overlapping sequence of max_length</li>
<li>Return the total number of rows in the dataset</li>
<li>Return a single row from the dataset</li>

<b> For efficient data loader implementation, we will use PyTorch's built-in dataset and dataloader classes.</b>

In [7]:
from torch.utils.data import Dataset
# First we will create a Dataset class that will be used by DataLoader to extract the data from 'raw_text' efficiently
class GPTDataset(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entier dataset
        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        # Use sliding window to chunk the book into overlapping sequence of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length - 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
        

In [8]:
# The above Dataset class will be served as input to create datasets using dataloader
from torch.utils.data import DataLoader
import tiktoken
def dataloader_v1(text, batch_size=4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    # Initialize tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataset(text=text, tokenizer=tokenizer, stride=stride, max_length=max_length)

    # Create Dataloader
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [9]:
# Convert dataloader into a python iterator to fetch next entry
import torch
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = dataloader_v1(text=raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)

print(next(data_iter))

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885]])]


In [ ]:
# Let's pull two other data and confirm that stride is one.
# The second element of first tensor should be first element of next tensor.
# In reality, low stride leads to overfitting since the overlapped data is fed multiple times
# It is good to have a stride = length of input vector
for i in range(2):
    print(next(data_iter))

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464]])]
[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807]])]


### Step 3: TOKEN EMBEDDING

In [14]:
# Create an embedding layer of vocab size and 256 length vector for each token
embedding_layer = torch.nn.Embedding(tokenizer.n_vocab, 256)
print(embedding_layer.weight.shape)

torch.Size([50257, 256])


In [34]:
# Now embedding layer has 50257x256
# Retrieve token embedding vector for raw text tokens
# We need to get input vectors as 8x4 --> 8 inputs with 4 token/input
max_length = 4
dataloader = dataloader_v1(text = raw_text, batch_size=8, max_length=max_length, stride=4, shuffle=False)
data_iter = iter(dataloader)
integers, _ = next(data_iter)
print(integers)
print(integers.shape)
embedding_vector = embedding_layer(torch.tensor(integers))
print(embedding_vector.shape)
print(embedding_vector)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
torch.Size([8, 4])
torch.Size([8, 4, 256])
tensor([[[ 0.7709,  0.0156, -0.6348,  ..., -1.4782,  0.9493,  1.8082],
         [-0.6345,  0.0218,  0.9705,  ..., -0.5795,  3.3748, -0.6350],
         [ 0.1139, -1.8634, -0.5039,  ..., -0.7990,  1.0489,  0.1555],
         [ 0.2059, -0.9976, -0.1893,  ...,  0.3613, -0.0398,  1.0546]],

        [[ 1.6244,  2.6661, -0.0340,  ...,  0.2866, -0.3518,  0.3947],
         [-0.9423,  0.6046,  0.0386,  ...,  0.2155, -0.0626,  0.8304],
         [-1.1962,  0.8808, -1.1823,  ...,  0.2864,  0.5574,  1.4500],
         [-0.5577,  2.1031, -0.3253,  ..., -0.4192,  0.8419, -0.4898]],

        [[-0.1929, -1.5585,  0.6899,  ..., -1.7189, -1.4585,  1.4830],
         

C:\Users\Adam\AppData\Local\Temp\ipykernel_33056\1738613274.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embedding_vector = embedding_layer(torch.tensor(integers))


### Step 4: POSITIONAL EMBEDDING (ENCODING WORD POSITIONS)

In [28]:
# In positional embedding, we will have another layer called token embedding layer like in token embedding vector
pos_embedding_layer = torch.nn.Embedding(max_length, 256) # Because we want just one input length, creates a lookup table 

print(pos_embedding_layer.weight.shape)
print(pos_embedding_layer.weight)


torch.Size([4, 256])
Parameter containing:
tensor([[ 0.1521,  0.6823, -0.0041,  ..., -0.7542, -1.6720,  1.7446],
        [ 0.0170,  0.1187,  0.5695,  ..., -0.0959,  0.1300,  1.3664],
        [ 0.1792,  0.9517,  0.0896,  ...,  1.2800, -1.1555, -0.4063],
        [-0.8819, -0.4398, -0.5946,  ...,  0.9804, -0.9617,  0.8497]],
       requires_grad=True)


In [29]:
# Just reshape
idx = torch.arange(max_length)
print(idx)
pos_embedding = pos_embedding_layer(idx)
print(pos_embedding.shape)
print(pos_embedding)

tensor([0, 1, 2, 3])
torch.Size([4, 256])
tensor([[ 0.1521,  0.6823, -0.0041,  ..., -0.7542, -1.6720,  1.7446],
        [ 0.0170,  0.1187,  0.5695,  ..., -0.0959,  0.1300,  1.3664],
        [ 0.1792,  0.9517,  0.0896,  ...,  1.2800, -1.1555, -0.4063],
        [-0.8819, -0.4398, -0.5946,  ...,  0.9804, -0.9617,  0.8497]],
       grad_fn=<EmbeddingBackward0>)


### Step 5: INPUT LAYER (TO FEED TO LLM FOR TRAINING)

In [38]:
input_layer = embedding_vector + pos_embedding
print(input_layer.shape)
print(input_layer)


torch.Size([8, 4, 256])
tensor([[[ 0.9230,  0.6979, -0.6388,  ..., -2.2324, -0.7226,  3.5528],
         [-0.6174,  0.1405,  1.5400,  ..., -0.6754,  3.5048,  0.7314],
         [ 0.2932, -0.9117, -0.4143,  ...,  0.4810, -0.1067, -0.2508],
         [-0.6760, -1.4374, -0.7839,  ...,  1.3416, -1.0015,  1.9043]],

        [[ 1.7765,  3.3485, -0.0380,  ..., -0.4676, -2.0238,  2.1393],
         [-0.9253,  0.7233,  0.6081,  ...,  0.1196,  0.0674,  2.1969],
         [-1.0169,  1.8325, -1.0928,  ...,  1.5663, -0.5981,  1.0437],
         [-1.4396,  1.6633, -0.9199,  ...,  0.5611, -0.1198,  0.3599]],

        [[-0.0408, -0.8761,  0.6859,  ..., -2.4731, -3.1304,  3.2276],
         [ 0.1202, -1.3449,  0.7388,  ..., -0.1229, -0.5863, -0.3193],
         [ 0.2365,  0.0854,  1.2205,  ...,  0.6920, -2.0288, -0.4767],
         [-1.8934, -1.7525, -2.8228,  ...,  0.0643, -1.2300,  0.6816]],

        ...,

        [[ 0.0605,  0.3928,  1.4615,  ...,  0.1355, -2.4694,  1.8392],
         [ 0.1604,  0.3839,  1.94